In [2]:
# Mount to Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define Project Folder
FOLDERNAME = 'Colab\ Notebooks/SC201L16'

%cd drive/MyDrive/$FOLDERNAME

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Mounted at /content/drive
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-da9e96b6d31b>", line 8, in <cell line: 8>
    get_ipython().run_line_magic('cd', 'drive/MyDrive/$FOLDERNAME')
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py", line 342, in cd
    oldcwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-p

In [3]:
# Define device
import torch
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print('Device:', device)

Device: cuda


In [4]:
# Load Existing Dataset
import torchvision.datasets as dset
import torchvision.transforms as T
transform = T.Compose([T.ToTensor(),T.RandomAutocontrast()])
train_data = dset.MNIST('./train',train=True,download=True,transform=transform)
val_data = dset.MNIST('./val',train=False,download=True,transform=transform)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-1815dd68026e>", line 5, in <cell line: 5>
    train_data = dset.MNIST('./train',train=True,download=True,transform=transform)
  File "/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py", line 99, in __init__
    self.download()
  File "/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py", line 179, in download
    os.makedirs(self.raw_folder, exist_ok=True)
  File "/usr/lib/python3.10/os.py", line 215, in makedirs
    makedirs(head, exist_ok=exist_ok)
  File "/usr/lib/python3.10/os.py", line 215, in makedirs
    makedirs(head, exist_ok=exist_ok)
  File "/usr/lib/python3.10/os.py", line 225, in makedirs
    mkdir(name, mode)
OSError: [Errno 107] Transport endpoint is not connected: './train'

During handling of the above exception, a

In [5]:
# Check Data Dimension
print(train_data)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./train
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               RandomAutocontrast(p=0.5)
           )


In [6]:

train_data[0][0].shape ## print data

torch.Size([1, 28, 28])

In [7]:
num_train = len(train_data)
num_val = len(val_data)
print('Number of training:', num_train)
print('Number of validation:', num_val)

Number of training: 60000
Number of validation: 10000


In [1]:
# Build Model
import torch.nn as nn
model = None
# model = nn.Sequential(
#     # N x 1 x 28 x 28

#     nn.Conv2d(in_channels=1,out_channels=64,kernel_size=3,padding=1,stride =1),
#     nn.BatchNorm2d(num_features=64),
#     nn.ReLU(),
#     nn.MaxPool2d(kernel_size=2,stride=2),
#     # 16 x 64 x 14 x 14

#     nn.Flatten(),
#     # 10 neurons(0-9)
#     nn.Linear(64*14*14,10)
# )
class MyModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv2d = nn.Conv2d(1,64,3,1,1)
    self.bn2d = nn.BatchNorm2d(64)
    self.fc = nn.Linear(64*14*14,10)

  def forward(self,x):
    out = self.conv2d(x)
    out = self.bn2d(out)
    out = nn.functional.relu(out)
    out = nn.functional.max_pool2d(out,(2,2))
    out = self.fc(out)
    return out
model = MyModel()

In [8]:
# Move model to GPU
# -> GPU
model = model.cuda()

In [9]:
# Create Mini-batches
from torch.utils.data import DataLoader
mini_trains = DataLoader(train_data, batch_size=128, shuffle=True)
mini_vals = DataLoader(val_data, batch_size=128, shuffle=True)

In [10]:
# Training Procedure
def train(num_epoch, model, mini_trains, mini_vals, device, loss_function, optimizer):
  for epoch in range(num_epoch):
    num_iters = 0
    for x, y in mini_trains:
      model.train()
      x = x.to(device)
      y = y.to(device)
      scores = model(x)
      loss = loss_function(scores, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if num_iters % 10 == 0:
        evaluate_predictor(model, epoch, mini_vals, device)
      num_iters += 1

In [11]:
# Validating Procedure
def evaluate_predictor(model, epoch, mini_vals, device):
  model.eval()
  with torch.no_grad():
    acc_count = 0
    for x, y in mini_vals:
      x = x.to(device)
      y = y.to(device)
      scores=model(x)
      predictions=scores.max(1)[1]
      acc = predictions.eq(y).sum().item()
      acc_count += acc
    print(f'Epoch[{epoch+1}] Acc: {acc_count/num_val}')

In [12]:
# Define loss function & optimizer
import torch.optim as optim
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [13]:
# Start training
train(1, model, mini_trains, mini_vals, device, loss_function, optimizer)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-df9119c3ec53>", line 2, in <cell line: 2>
    train(1, model, mini_trains, mini_vals, device, loss_function, optimizer)
  File "<ipython-input-21-130213b07839>", line 9, in train
    scores = model(x)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "<ipython-input-18-7d4a812f56f8>", line 29, in forward
    out = self.fc(out)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-pac